In [3]:
import math
import numpy as np
import sympy

In [2]:
#for primitive functions use sympy (i.e., sympy.cos() for cos())
def func(x):
    return x+sympy.sin(x)-1
def function2(x):
    return (x+18)**(0.5)+1-x
def function2_der(x):
    return 0.5*(x+18)**(-0.5)-1
def testfunc(x):
    return x**3-3*x**2-1-x
def testfunc_der(x):
    return 3*x**2-6*x-1

## [Fixed-point iteration method](https://en.wikipedia.org/wiki/Fixed-point_iteration)


In [3]:
def fpi(function,eps,x0):
    """
     Arguments:
        function -- equation,defined above
        eps -- const to satisfy: abs(f(c)-f(a))>=eps
        x0 -- starting x0
        Output:
        c -- equation root
        n -- number of iterations
    """
    n,x=0,x0,
    x1 = function(x)
    while(abs(x-x1)>=eps):
        n+=1
        x = x1
        x1 = function(x)

        if n%100==0:
            print(n)
    return x,n   

In [4]:
fpi(function2,0.0001,0)

(2.7791609465066296, 94)

## [Bisection method](https://en.wikipedia.org/wiki/Bisection_method)

In [5]:
def bisection(function, a,b, eps):
    """
        Arguments:
        function -- equation, defined above
        a,b  -- bounds
        eps -- precision
        Output:
        c -- equation root
        n -- number of iterations
        
    """
    def iterate():
        """
        Output:
        c -- new center of a section 
        y -- function value at new c
        """
        c = (a+b)/2.0
        y = function(c)
        return c,y
    c,y = iterate()
    n = 0
    while (y!=0 and abs(function(c)-function(a))>=eps):
        if (np.sign(y))==np.sign(function(a)):
            a=c
        else:
            b=c
        c,y = iterate()
        n+=1
        if n%100==0:
            print(n)
    return c,n

In [6]:
bisection(function2,5,6,0.0001)

(5.88751220703125, 13)

## [Newton's method](https://en.wikipedia.org/wiki/Newton%27s_method)

In [7]:
def newton_nosympy(function, derivative, x0, eps):
    """
    Arguments:
    ---------
    function - given equation
    derivative - derivative of function of given equation
    x0 - initial guess
    eps - precision
    """
    x1,x0,n = x0,function(x0),0
    while (abs(x1-x0)>=eps):
        x1,x0 = x1-function(x1)/derivative(x1),x1
        n += 1
        if n%1000==0:
            return "failure"
    return x1,n

In [8]:
def testfunc(x):
    return x**2-20*math.sin(x)
def testfunc_deriv(x):
    return 2*x-20*math.cos(x)
newton_nosympy(testfunc,testfunc_deriv,2,0.000001)

(2.752946633818705, 5)

In [9]:
def newton(function, x0, eps):
    """
    Arguments:
    function -- given equation
    x0 - initial guess
    eps - precision
    Output:
    x -- root of the equation
    n -- number of iterations
    """
    n,x0,x1 = 0,x0,0
    
    x = sympy.Symbol('x')#variable
    f = function(x)
    
    derivative = sympy.diff(f)
    
    print(derivative)
    
    x1-= float(f.subs({'x':x0}))/float(derivative.subs({'x':x0}))
    while(abs(x1-x0)>=eps):
        #x0=x1
        x1=x0 - float(function(x0))/float(derivative.subs({'x':x0})) 
        x0=x1
        n+=1
        if n%10==0:
            print("n: {0}, |x1-x0|={1}".format(n,abs(x1-x0)))
    return x1,n

### [Gaussian elimination](https://en.wikipedia.org/wiki/Gaussian_elimination)

In [10]:
def gaussian(A,b):
    """
    Arguments:
    A -- numpy.ndarray -- given system of coeffitients
    x -- numpy.ndarray -- unknown vector X 
    b -- vector b
    Output:
    x -- root of the system of equations A*x=b
    """
    n,matr = len(b),np.hstack([A,b.reshape(-1,1)])
    if np.linalg.det(A)==0: 
        return -1
    #direct move
    for i in range(n):
        a = matr[i]
        for j in range(i+1,n):
            b = matr[j]
            m = a[i]/b[i]
            matr[j] = a - m*b
            
    #reversed move
    for i in range(n-1,-1,-1):
        matr[i]= matr[i]/matr[i,i]
        a = matr[i]
        for j in range(i-1,-1,-1):
            b = matr[j]
            m = a[i]/b[i]
            matr[j] = a - m*b

    return matr[:,3]

In [11]:
A = np.array([[1, 2, 3], [3, 5, 7], [1, 3, 4]], dtype='float')
b = np.array([3, 0, 1])
print(gaussian(A,b))
t1,t2 = gaussian(A,b),np.linalg.solve(A,b)
t1 = np.round(t1,10)
t2 = np.round(t2,10)
np.testing.assert_array_equal(t1,t2)

[ -4. -13.  11.]


### [Tridiagonal matrix algorithm](https://google.com)

In [22]:
def get_coeff_vect(array):
    """
    Arguments:
    ----------
    array -- array of shape(n,n)
    returns triagonals a,b,c
    Output:
    a,b,c -- tridiagonals of an array
    """
    n = len(array)
    ind1,ind2 = [i for i in range(1,n)],[i-1 for i in range(1,n)]
    a = np.copy(array[ind1,ind2])
    b = np.copy(array[[i for i in range(n)],[i for i in range(n)]])
    c = np.copy(array[ind2,ind1])
    return a,b,c


def tridiagonal_matr_algo(arr,f,local_coeff=False):
    """
    Arguments:
    ----------
    arr -- array of shape(n,n)
    f -- vector of shape(n,)
    local_coeff -- bool, uses slighty different calculations
    Output:
    x -- solution of system of equations A*x = f
    """
    n,f= len(arr),np.copy(f)  
    a,b,c = get_coeff_vect(arr)
  
    if local_coeff:
        alpha,beta = [],[]
        alpha.append(c[0]/b[0])
        beta.append(f[0]/b[0])
        
        #forward step
        for i in range(1,n):
            if i!=n-1:
                alpha.append(c[i]/(b[i]-alpha[-1]*a[i-1]))
            beta.append((f[i]-beta[-1]*a[i-1])/(b[i]-alpha[-1]*a[i-1]))

        
        #backward step
        x = beta
        for i in reversed(range(n-1)):
            x[i] = beta[i]-alpha[i]*x[i+1]
        
        return x
    else: 
        # forward step
        for i in range(1,n):
            w = a[i-1]/b[i-1]
            b[i] -= w*c[i-1]
            f[i] -= w*f[i-1]
        #backward step
        x = b
        x[-1] = f[-1]/b[-1]
        for i in range(n-2,-1,-1):
            x[i]=(f[i]-c[i]*x[i+1])/(b[i])
        return x   

In [20]:
A,d = np.array([[-4,1,0,0],[1,-4,-1,0],[0,1,-4,1],[0,0,1,-4]],dtype=float),np.array([-6,-4,-4,-6],dtype=float)

In [23]:
C,d = np.array([[-4,1,0,0],[1,-4,1,0],[0,1,-4,1],[0,0,1,-4]],dtype=float),np.array([-6,-4,-4,-6],dtype=float)
print("TMA, no local coefficient preservation: {0}".format(tridiagonal_matr_algo(C,d)))
print("TMA, local coefficient preservation:{0}".format(tridiagonal_matr_algo(C,d,True)))
print("Standard Gaussian solver: {0}".format(np.linalg.solve(C,d)))

TMA, no local coefficient preservation: [2. 2. 2. 2.]
TMA, local coefficient preservation:[2.001795631286786, 2.0071825251471433, 2.002380897873217, 2.000595224468304]
Standard Gaussian solver: [2. 2. 2. 2.]


### [Seidel method](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method)

$Ax = b$<br>

$L_* $ - lower triangular component of A <br>
$U $  - strict upper triangular component of A<br><br>
$T = -L_*^{-1}U$<br>
$C=L_*^{-1}b$<br>

iteratively, $x^{(k+1)}=Tx^{(k)}+C$

In [34]:
def seidel(A,b,x0,eps):
    """
    Arguments:
    A -- array of shape(n,n)
    b -- array of shape(n,1)
    x0 -- initial guess - array of shape(n)
    eps -- precision
    Returns:
    x1 -- array of shape(n,)
    n -- number of iterations
    """
    L,U = np.tril(A),np.triu(A,1)
    L_inv = np.linalg.inv(L)
    
    T,C = np.dot(-L_inv,U), np.dot(L_inv,b)
    
    n, x1 = 0, np.dot(T,x)+C
    while not np.allclose(x0,x1,rtol=eps):
        x0 = x1
        x1 = np.dot(T,x0)+C
        n+=1
        if n==1000:
            return "smth went wrong"
    return x1,n

In [39]:
A,b = np.array([[16,3],[7,-11]]),np.array([11,13])
x = np.array([1,1],dtype=float)
seidel(A,b,x,1e-8)

(array([ 0.81218274, -0.66497462]), 9)

In [40]:
C,d = np.array([[2/3,1/6,0],[1/6,2/3,1/6],[0,1/6,2/3]],dtype=float),np.array([5/6,1,5/6]) 
x = np.array([0,0,0],dtype=float)
seidel(C,d,x,1e-7)

(array([1., 1., 1.]), 9)